# NLP

In [7]:
# === Anchor-based, single-CSV pipeline (YAML-faithful + acronym-safe) ===

import os, re, json, yaml, string, difflib, bisect
from typing import Dict, Iterable, List, Tuple, Optional
from dataclasses import dataclass
from collections import Counter
import pandas as pd
from tqdm import tqdm

# ---------------- NLTK bootstrap ----------------
import nltk
def _ensure_nltk():
    for res, pkg in [
        ("tokenizers/punkt","punkt"),
        ("corpora/wordnet","wordnet"),
        ("corpora/omw-1.4","omw-1.4"),
    ]:
        try: nltk.data.find(res)
        except LookupError: nltk.download(pkg, quiet=True)
_ensure_nltk()
from nltk.stem import WordNetLemmatizer
_lem = WordNetLemmatizer()

# ---------------- Normalisation & compounding ----------------
def load_cpwords(py_path: str) -> Dict[str,str]:
    env = {}
    with open(py_path, "r", encoding="utf-8") as f:
        code = f.read()
    exec(compile(code, py_path, "exec"), env, env)
    if "compound_keywords" not in env or not isinstance(env["compound_keywords"], dict):
        raise ValueError("compound_keywords not found in cpwords file")
    out = {}
    for k, v in env["compound_keywords"].items():
        kk = str(k).strip().lower()
        vv = str(v).strip().lower().replace("-", " ").replace(" ", "_")
        if kk: out[kk] = vv
    return out

def _collect_multiword_phrases_from_ontology(ont: dict) -> List[str]:
    out = set()
    def add_if_multi(s):
        s = str(s).strip()
        if s and (" " in s or "-" in s):
            out.add(s.lower())
    for v in (ont or {}).values():
        if isinstance(v, dict):
            for vv in v.values():
                if isinstance(vv, (list, tuple, set)):
                    for t in vv: add_if_multi(t)
                else:
                    add_if_multi(vv)
        elif isinstance(v, (list, tuple, set)):
            for t in v: add_if_multi(t)
        else:
            add_if_multi(v)
    return sorted(out, key=len, reverse=True)

def compile_compounds_regex(phrases: Iterable[str], cpwords_map: Dict[str,str]) -> re.Pattern:
    base = set(p.strip().lower() for p in phrases if p and p.strip())
    base |= set(k.strip().lower() for k in (cpwords_map or {}).keys())
    if not base: return re.compile(r"(?!x)x")
    toks = sorted(base, key=len, reverse=True)
    pat = r"\b(" + "|".join(re.escape(p) for p in toks) + r")\b"
    return re.compile(pat, re.I)

def apply_compounds(text: str, compounds_rx: re.Pattern, cpwords_map: Dict[str,str]) -> str:
    def repl(m):
        key = m.group(1).lower()
        return cpwords_map.get(key, key.replace("-", "_").replace(" ", "_"))
    return compounds_rx.sub(repl, text)

def canonicalise_term(term: str) -> str:
    t = term.lower().strip().replace("-", "_").replace(" ", "_")
    return re.sub(r"_+", "_", t)

def normalize(text: str, compounds_rx: Optional[re.Pattern], cpwords_map: Dict[str,str]) -> str:
    if not text: return ""
    t = text.lower()
    if compounds_rx is not None:
        t = apply_compounds(t, compounds_rx, cpwords_map)
    keep_us = string.punctuation.replace("_","")
    t = re.sub(rf"[{re.escape(keep_us)}]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    t = " ".join(_lem.lemmatize(w) for w in t.split())
    return t

# ---------------- Sentence split & token windows ----------------
_SENT_SPLIT = re.compile(r'(?<=[\.\?\!])\s+')
def sents(text: str) -> List[str]:
    try:
        from nltk.tokenize import sent_tokenize
        return sent_tokenize(text or "")
    except Exception:
        return _SENT_SPLIT.split(text or "")

def tokenize(text: str):
    tokens, starts = [], []
    for m in re.finditer(r"\S+", text):
        tokens.append(m.group(0))
        starts.append(m.start())
    return tokens, starts

def window_from_charspan(text: str, tokens: List[str], starts: List[int], span: Tuple[int,int], k: int) -> str:
    i = bisect.bisect_right(starts, span[0]) - 1
    if i < 0: i = 0
    lo = max(0, i - k); hi = min(len(tokens)-1, i + k)
    return " ".join(tokens[lo:hi+1])

# ---------------- Regex helpers ----------------
SEPARATOR_FLEX = r"[ _-]+"
def _flexify(term: str) -> str:
    t = (term or "").strip()
    t = re.escape(t)
    t = re.sub(r"\\\s+", SEPARATOR_FLEX, t)
    return t + r"\w*"

def build_regexes_from_terms(terms: Iterable[str], require_min_ngram=None) -> List[re.Pattern]:
    regs, seen = [], set()
    for t in (terms or []):
        v = canonicalise_term(t)
        if not v: continue
        if require_min_ngram and len(re.split(r"[ _-]+", v)) < require_min_ngram:
            continue
        key = ("rx", v)
        if key in seen: continue
        seen.add(key)
        regs.append(re.compile(r"\b" + _flexify(v) + r"\b", re.I))
    return regs

# ---------------- Action/performance gates ----------------
VERBS_RX     = re.compile(r"\b(model(?:ing|led|ed|s)?|estimate(?:d|s|r)?|predict(?:ed|s|ive)?|analy[sz]e(?:d|s|r)?|simulate(?:d|s)?)\b", re.I)
FAVOURED_RX  = re.compile(r"\b(performance|accuracy|efficienc\w*|load|consumption|demand|baseline|benchmark(?:ing)?|error|mae|rmse|mape)\b", re.I)
MODEL_WORD_RX= re.compile(r"\bmodel(?:s|ing|led|ed)?\b", re.I)
GENERIC_OPT_RX = re.compile(r"\b(optimi[sz]e?(?:d|s|r)?|optimi[sz]ation|multi-?objective)\b", re.I)

# ---------------- Gappy “<term> … model” support ----------------
GAP_MAX   = 3   # words allowed between term and "model"
WORD_WINDOW = 15# ± words around anchor
MERGE_SIM = 0.90# similarity threshold for merging

TERM_SUFFIX_RX = r"(?:-?(?:based|driven|guided|assisted|enabled|oriented|like|alike))?"

def _sep_flexify(s: str) -> str:
    s = canonicalise_term(s)
    parts = [re.escape(p) for p in s.split("_") if p]
    if not parts: return ""
    return r"(?:%s)" % r"[ _-]+".join(parts)

def build_gappy_model_regexes(terms: Iterable[str], gap_max: int = GAP_MAX) -> List[re.Pattern]:
    regs = []
    for t in set(canonicalise_term(x) for x in terms if str(x).strip()):
        core = _sep_flexify(t)
        if not core: continue
        pat = r"\b" + core + TERM_SUFFIX_RX + r"(?:\W+\w+){0," + str(gap_max) + r"}\W+model\w*\b"
        regs.append(re.compile(pat, re.I))
    return regs

def any_regex_hits(text: str, regs: List[re.Pattern]) -> List[str]:
    out = []
    for rx in regs:
        out += [m.group(0) for m in rx.finditer(text)]
    return out

def extract_term_from_gappy_hit(hit: str) -> str:
    s = hit.lower()
    s = s.split(" model", 1)[0]
    s = re.sub(r"(?:\W+\w+){0," + str(GAP_MAX) + r"}$", "", s)  # trim trailing gap words
    s = re.sub(TERM_SUFFIX_RX + r"$", "", s)                    # strip suffix
    return canonicalise_term(s)

# ---------------- 90% merge for near-duplicates ----------------
def _sim(a: str, b: str) -> float:
    return difflib.SequenceMatcher(None, a, b).ratio()

def merge_counter_by_similarity(counter: "Counter[str]", threshold: float = MERGE_SIM) -> "Counter[str]":
    items = [(canonicalise_term(k), v) for k, v in counter.items()]
    items.sort(key=lambda kv: len(kv[0]))
    reps, used = [], set()
    for i, (k, v) in enumerate(items):
        if i in used: continue
        cluster = [(k, v)]; used.add(i)
        for j in range(i+1, len(items)):
            if j in used: continue
            k2, v2 = items[j]
            if _sim(k, k2) >= threshold or k in k2 or k2 in k:
                cluster.append((k2, v2)); used.add(j)
        rep = min((t for t,_ in cluster), key=len)
        reps.append((rep, sum(c for _, c in cluster)))
    out = Counter()
    for k, v in reps: out[k] += v
    return out

def merge_set_by_similarity(terms: Iterable[str], threshold: float = MERGE_SIM) -> List[str]:
    cnt = Counter(canonicalise_term(t) for t in terms if str(t).strip())
    merged = merge_counter_by_similarity(cnt, threshold)
    return sorted(merged.keys())

# ---------------- Optional Word2Vec expansion (scale only) ----------------
try:
    from gensim.models import Word2Vec as GensimWord2Vec
except Exception:
    GensimWord2Vec = None

@dataclass
class NLPConfig:
    mode: str = "auto"   # off|light|full|auto
    model_path: str = ""
    exp_topn: int = 40
    exp_thresh: float = 0.70
    exp_min_count: int = 10
    context_k: int = 1

def load_nlp_config(path_or_ontology_yaml: Optional[str]) -> NLPConfig:
    cfg = NLPConfig()
    if path_or_ontology_yaml and os.path.exists(path_or_ontology_yaml):
        with open(path_or_ontology_yaml, "r", encoding="utf-8") as f:
            y = yaml.safe_load(f) or {}
        y = y.get("_nlp", y)
        cfg.mode = str(y.get("mode", cfg.mode)).lower()
        cfg.model_path = str(y.get("model_path", cfg.model_path))
        cfg.exp_topn = int(y.get("exp_topn", cfg.exp_topn))
        cfg.exp_thresh = float(y.get("exp_thresh", cfg.exp_thresh))
        cfg.exp_min_count = int(y.get("exp_min_count", cfg.exp_min_count))
        cfg.context_k = int(y.get("context_k", cfg.context_k))
    return cfg

def resolve_nlp_mode(cfg: NLPConfig) -> str:
    if cfg.mode in {"off","light","full"}: return cfg.mode
    if GensimWord2Vec is None: return "light"
    candidates = [cfg.model_path] if cfg.model_path else []
    candidates += ["model.word2vec","model.w2v","word2vec.model", os.path.join(os.getcwd(),"word2vec.model")]
    for p in candidates:
        if p and os.path.exists(p):
            try:
                GensimWord2Vec.load(p); return "full"
            except Exception:
                continue
    return "light"

def expand_terms_w2v(model, seeds: List[str], topn=40, thresh=0.70, min_count=10) -> List[str]:
    out = set(canonicalise_term(s) for s in seeds if s)
    if not model: return sorted(out)
    for t in list(out):
        if t in model.wv:
            for cand, sim in model.wv.most_similar(t, topn=topn):
                if sim < thresh: continue
                try:
                    if model.wv.get_vecattr(cand,"count") < min_count: continue
                except Exception: pass
                if any(ch.isdigit() for ch in cand): continue
                out.add(canonicalise_term(cand))
    return sorted(out)

# ---------------- IO ----------------
def load_jsons(dirpath):
    for fn in os.listdir(dirpath):
        if fn.endswith(".json"):
            p = os.path.join(dirpath, fn)
            try:
                with open(p, "r", encoding="utf-8") as f:
                    yield fn, json.load(f)
            except Exception as e:
                print(f"[WARN] failed to read {fn}: {e}")

# ---------------- Acronym-safe matchers (use RAW text) ----------------
_ACRO_LEFT  = r"(?<![A-Za-z0-9_])"
_ACRO_RIGHT = r"(?![A-Za-z0-9_])"

def pick_acronyms(terms: Iterable[str]) -> List[str]:
    acr = []
    for t in (terms or []):
        s = str(t).strip()
        if not s: continue
        if " " in s: continue          # spaces → phrase, not acronym
        # keep shortish tokens or mixed with digits/dots (e.g., PM2.5)
        if len(s) <= 6 or re.search(r"[0-9.]", s):
            acr.append(s)
    return acr

def build_acronym_regexes(terms: Iterable[str]) -> List[re.Pattern]:
    regs = []
    for t in pick_acronyms(terms):
        pat = _ACRO_LEFT + re.escape(t.upper()) + _ACRO_RIGHT
        regs.append(re.compile(pat))    # case-sensitive on raw text
    return regs

def count_yaml_category(raw_text: str, norm_text: str, terms: Iterable[str]) -> Counter:
    """
    Count matches for a YAML term list by combining:
      - phrase tolerant matches on normalized text (lowercased)
      - acronym strict matches on raw text (case-sensitive, whole-token)
    """
    cnt = Counter()
    # phrases on normalized text
    phrase_rx = build_regexes_from_terms(terms, require_min_ngram=1)
    for rx in phrase_rx:
        for m in rx.finditer(norm_text):
            cnt[canonicalise_term(m.group(0))] += 1
    # acronyms on raw text
    acro_rx = build_acronym_regexes(terms)
    for rx in acro_rx:
        for m in rx.finditer(raw_text):
            cnt[canonicalise_term(m.group(0))] += 1
    return merge_counter_by_similarity(cnt, threshold=MERGE_SIM)

# ---------------- Paradigm term-only regex builder ----------------
def build_simple_paradigm_term_rx(paradigms: dict) -> dict:
    out = {}
    for pname, terms in (paradigms or {}).items():
        canon_terms = [canonicalise_term(t) for t in (terms or [])]
        out[pname] = build_regexes_from_terms(canon_terms, require_min_ngram=1)
    return out

# ---------------- Core analysis (anchor-based + YAML-faithful cats) ----------------
def analyze_anchor_based_single(doc: dict,
                                ont: dict,
                                compounds_rx: Optional[re.Pattern],
                                cp_map: Dict[str,str],
                                cfg: NLPConfig,
                                w2v) -> dict:
    raw = (doc.get("full-text-retrieval-response", {}) or {}).get("originalText","") or ""
    text = normalize(raw, compounds_rx, cp_map)  # normalized (lowercased)
    S = sents(text)
    tokens, starts = tokenize(text)

    # Ontology buckets
    scale       = ont.get("scale", {}) or {}
    paradigms   = ont.get("model_paradigm", {}) or {}
    optim_cats  = ont.get("optimization_methods", {}) or {}
    app_cats    = ont.get("applications", {}) or {}
    data_cats   = ont.get("data_types", {}) or {}

    # Canonicalised seeds (+ optional W2V) for scale only
    def seeds(key):
        base = [canonicalise_term(x) for x in (scale.get(key, []) or [])]
        return expand_terms_w2v(w2v, base, cfg.exp_topn, cfg.exp_thresh, cfg.exp_min_count) if w2v else base

    building_terms = seeds("building_model")
    system_terms   = seeds("system_model")
    climate_terms  = seeds("climate_model")
    occup_terms    = seeds("occupancy_model")

    # Finders for scale terms (need spans)
    def rx_find_terms(terms):
        if not terms: return None
        pats = [r"\b" + _flexify(t) + r"\b" for t in set(terms)]
        return re.compile("|".join(pats), re.I)

    RX_FIND = {
        "building": rx_find_terms(building_terms),
        "system":   rx_find_terms(system_terms),
        "weather":  rx_find_terms(climate_terms),
        "occupancy":rx_find_terms(occup_terms),
    }

    # Paradigm & optimisation matchers
    par_rx        = {k: build_regexes_from_terms(v, require_min_ngram=1) for k, v in (paradigms or {}).items()}
    par_gappy_rx  = {k: build_gappy_model_regexes(v, gap_max=GAP_MAX)   for k, v in (paradigms or {}).items()}
    par_term_rx   = build_simple_paradigm_term_rx(paradigms)
    opt_rx        = {k: build_regexes_from_terms(v, require_min_ngram=1) for k, v in (optim_cats or {}).items()}

    # Containers
    term_counts = {c: Counter() for c in RX_FIND.keys()}
    par_found   = {c: set() for c in RX_FIND.keys()}
    par_terms   = {c: set() for c in RX_FIND.keys()}
    opt_methods, opt_terms = set(), set()

    # PRIMARY path — scale-term windows
    for cat, rx in RX_FIND.items():
        if rx is None: continue
        for m in rx.finditer(text):
            w = window_from_charspan(text, tokens, starts, m.span(), k=WORD_WINDOW)
            if not (VERBS_RX.search(w) or FAVOURED_RX.search(w)):
                continue
            term_counts[cat][canonicalise_term(m.group(0))] += 1

            # Paradigms in this window → ONLY this category
            for pname in par_rx.keys():
                plain_hits = []
                for rxx in par_rx[pname]:
                    plain_hits += [canonicalise_term(h.group(0)) for h in rxx.finditer(w)]
                gappy_strings = any_regex_hits(w, par_gappy_rx.get(pname, []))
                gappy_terms   = [extract_term_from_gappy_hit(h) for h in gappy_strings]
                belongs_via_gappy = False
                if gappy_terms and par_term_rx.get(pname):
                    for t_ in gappy_terms:
                        if any(r.search(t_) for r in par_term_rx[pname]):
                            belongs_via_gappy = True; break
                if plain_hits or belongs_via_gappy:
                    par_found[cat].add(pname)
                    par_terms[cat].update(plain_hits)
                    par_terms[cat].update(gappy_terms)

            # Optimisation phrases in this window (global list)
            for oname, regs in opt_rx.items():
                found = []
                for rxx in regs:
                    found += [canonicalise_term(h.group(0)) for h in rxx.finditer(w)]
                if found:
                    opt_methods.add(oname); opt_terms.update(found)
            g = GENERIC_OPT_RX.search(w)
            if g:
                opt_methods.add("generic"); opt_terms.add(canonicalise_term(g.group(0)))

    # FLEX path — paradigm anchored; map to categories if window shows 'model' + category hints
    CATEGORY_HINT = {
        "building":  re.compile(r"\b(building|facility|premise|asset|whole[_ ]building|zone|envelope|bem|ubem)\b", re.I),
        "system":    re.compile(r"\b(system|hvac|ahu|doas|vav|fcu|coil|chiller|boiler|pump|tower|vr[fb]|heat[_ ]pump)\b", re.I),
        "occupancy": re.compile(r"\b(occupanc\w*|occupant\w*|people|tenant|user)\b", re.I),
        "weather":   re.compile(r"\b(weather|climate|outdoor|ambient|meteorolog\w*)\b", re.I),
    }
    ALL_PAR_RXS = [r for regs in par_rx.values() for r in regs]

    for rx in ALL_PAR_RXS:
        for m in rx.finditer(text):
            w = window_from_charspan(text, tokens, starts, m.span(), k=WORD_WINDOW)
            if not (MODEL_WORD_RX.search(w) and (VERBS_RX.search(w) or FAVOURED_RX.search(w))):
                continue
            for cat in ["building","system","occupancy","weather"]:
                hint_ok  = CATEGORY_HINT[cat].search(w) is not None
                scale_ok = (RX_FIND[cat] is not None) and (RX_FIND[cat].search(w) is not None)
                if not (hint_ok or scale_ok): continue

                pstr = canonicalise_term(m.group(0))
                all_gappy_strings = any_regex_hits(w, [rgx for lst in par_gappy_rx.values() for rgx in lst])
                ext_terms = [extract_term_from_gappy_hit(h) for h in all_gappy_strings]

                matched_buckets = set()
                for pname, regs in par_rx.items():
                    if any(r.search(m.group(0)) for r in regs):
                        matched_buckets.add(pname)
                for pname, regs in par_term_rx.items():
                    if any(r.search(t_) for r in regs for t_ in ([pstr] + ext_terms)):
                        matched_buckets.add(pname)

                for pname in matched_buckets:
                    par_found[cat].add(pname)
                par_terms[cat].add(pstr)
                par_terms[cat].update(ext_terms)

    # Merge near-duplicates (scale/paradigm terms)
    for cat in term_counts.keys():
        term_counts[cat] = merge_counter_by_similarity(term_counts[cat], threshold=MERGE_SIM)
        par_terms[cat]   = set(merge_set_by_similarity(par_terms[cat], threshold=MERGE_SIM))

    # ---------------- YAML-faithful counts for optimization/applications/data_types ----------------
    # We do these at document-level (no window), with acronym-safe matching.
    opt_ranked_cols = {}
    for cat_name, terms in (optim_cats or {}).items():
        c = count_yaml_category(raw, text, terms)     # raw=acronyms; text=phrases
        opt_ranked_cols[f"optimization_{canonicalise_term(cat_name)}_ranked"] = (
            "NM" if not c else ";".join(f"{t} ({n})" for t, n in sorted(c.items(), key=lambda kv:(-kv[1], kv[0])))
        )

    app_ranked_cols = {}
    for cat_name, terms in (app_cats or {}).items():
        c = count_yaml_category(raw, text, terms)
        app_ranked_cols[f"applications_{canonicalise_term(cat_name)}_ranked"] = (
            "NM" if not c else ";".join(f"{t} ({n})" for t, n in sorted(c.items(), key=lambda kv:(-kv[1], kv[0])))
        )

    data_ranked_cols = {}
    for cat_name, terms in (data_cats or {}).items():
        c = count_yaml_category(raw, text, terms)
        data_ranked_cols[f"data_{canonicalise_term(cat_name)}_ranked"] = (
            "NM" if not c else ";".join(f"{t} ({n})" for t, n in sorted(c.items(), key=lambda kv:(-kv[1], kv[0])))
        )

    # Assemble row
    def ranked_pairs(counter: Counter) -> str:
        if not counter: return "NM"
        items = sorted(counter.items(), key=lambda kv: (-kv[1], kv[0]))
        return ";".join(f"{t} ({c})" for t, c in items)

    def dedup_join(xs: Iterable[str]) -> str:
        vals = [canonicalise_term(x) for x in (xs or []) if str(x).strip()]
        return ";".join(sorted(set(vals))) if vals else "NM"

    row = {
        "building_terms_ranked":  ranked_pairs(term_counts["building"]),
        "system_terms_ranked":    ranked_pairs(term_counts["system"]),
        "weather_terms_ranked":   ranked_pairs(term_counts["weather"]),
        "occupancy_terms_ranked": ranked_pairs(term_counts["occupancy"]),
        "building_paradigms":       dedup_join(par_found["building"]),
        "system_paradigms":         dedup_join(par_found["system"]),
        "weather_paradigms":        dedup_join(par_found["weather"]),
        "occupancy_paradigms":      dedup_join(par_found["occupancy"]),
        "building_paradigm_terms":  dedup_join(par_terms["building"]),
        "system_paradigm_terms":    dedup_join(par_terms["system"]),
        "weather_paradigm_terms":   dedup_join(par_terms["weather"]),
        "occupancy_paradigm_terms": dedup_join(par_terms["occupancy"]),
        "optimization_methods":     dedup_join(opt_methods),
        "optimization_method_terms":dedup_join(opt_terms),
    }
    row.update(opt_ranked_cols)
    row.update(app_ranked_cols)
    row.update(data_ranked_cols)
    return row

# ---------------- Runner (writes a single CSV) ----------------
def run_anchor_based(
    input_dir: str,
    ontology_path: str,
    cpwords_path: Optional[str] = None,
    anchor_csv: str = "nlp_output/anchor_based.csv",
    nlp: str = "auto",
    config_path: Optional[str] = None,
    verbose: bool = True,
) -> pd.DataFrame:
    with open(ontology_path, "r", encoding="utf-8") as f:
        ont = yaml.safe_load(f) or {}
    cfg = load_nlp_config(config_path or ontology_path)
    if nlp: cfg.mode = nlp
    mode = resolve_nlp_mode(cfg)
    if verbose: print(f"[NLP] mode = {mode}")

    # Word2Vec (full)
    w2v = None
    if mode == "full" and GensimWord2Vec is not None:
        candidates = [cfg.model_path] if cfg.model_path else []
        candidates += ["model.word2vec","model.w2v","word2vec.model", os.path.join(os.getcwd(),"word2vec.model")]
        for p in candidates:
            if p and os.path.exists(p):
                try:
                    w2v = GensimWord2Vec.load(p)
                    if verbose: print(f"[NLP] loaded Word2Vec: {p}")
                    break
                except Exception as e:
                    if verbose: print(f"[WARN] could not load {p}: {e}")
        if w2v is None:
            if verbose: print("[WARN] falling back to light mode")
            mode = "light"

    # Compounding sources
    cp_map = {}
    if cpwords_path and os.path.exists(cpwords_path):
        try: cp_map = load_cpwords(cpwords_path)
        except Exception as e:
            if verbose: print(f"[WARN] cpwords not loaded: {e}")
    ont_multi = _collect_multiword_phrases_from_ontology(ont)
    compounds_rx = compile_compounds_regex(ont_multi, cp_map)

    # Process
    rows = []
    os.makedirs(os.path.dirname(anchor_csv) or ".", exist_ok=True)
    for fname, doc in tqdm(list(load_jsons(input_dir)), desc="Anchor-based"):
        r = analyze_anchor_based_single(doc, ont, compounds_rx, cp_map, cfg, w2v)
        r["file"] = fname
        rows.append(r)

    # Collect dynamic YAML-driven columns
    dyn_cols = []
    for grp_key in ("optimization_methods","applications","data_types"):
        for cat_name in (ont.get(grp_key, {}) or {}).keys():
            prefix = "optimization" if grp_key=="optimization_methods" else ("applications" if grp_key=="applications" else "data")
            dyn_cols.append(f"{prefix}_{canonicalise_term(cat_name)}_ranked")

    cols_fixed = [
        "file",
        "building_terms_ranked","system_terms_ranked","weather_terms_ranked","occupancy_terms_ranked",
        "building_paradigms","system_paradigms","weather_paradigms","occupancy_paradigms",
        "building_paradigm_terms","system_paradigm_terms","weather_paradigm_terms","occupancy_paradigm_terms",
        "optimization_methods","optimization_method_terms",
    ]
    cols = cols_fixed + dyn_cols
    df = pd.DataFrame(rows)
    # ensure all columns exist even if empty
    for c in cols:
        if c not in df.columns:
            df[c] = "NM"
    df = df[cols].sort_values("file")
    df.to_csv(anchor_csv, index=False)
    if verbose: print(f"[OK] wrote {anchor_csv} ({len(df)} rows)")
    return df


In [8]:

df_summary = run_anchor_based(
    input_dir="elsevier_out/papers_ftrr",
    ontology_path="nlp_input/ontology.yaml",
    cpwords_path="nlp_input/_cpwords.py",
    anchor_csv="nlp_output/anchor_summary.csv",
    nlp="auto",                              # off | light | full | auto
    config_path="nlp_input/ontology.yaml",                        # or a YAML with an _nlp block
)


[NLP] mode = full
[NLP] loaded Word2Vec: word2vec.model


Anchor-based: 100%|██████████| 14/14 [00:05<00:00,  2.58it/s]

[OK] wrote nlp_output/anchor_summary.csv (14 rows)


In [9]:
df_summary

,file,building_terms_ranked,system_terms_ranked,weather_terms_ranked,occupancy_terms_ranked,building_paradigms,system_paradigms,weather_paradigms,occupancy_paradigms,building_paradigm_terms,...,applications_energy_flexibility_ranked,applications_performance_tracking_ranked,applications_performance_analysis_ranked,data_metering_ranked,data_air_temperature_ranked,data_air_quality_ranked,data_environment_ranked,data_occupancy_ranked,data_hvac_signals_ranked,data_thermal_comfort_ranked
0,10.1016_j.apenergy.2019.113920.json,room (21);building (10);temperature (4),acmv (19);ahu (17);coil (17);doas (11);chiller...,NM,internal (4);occupancy (3),blackbox;greybox,blackbox;greybox,NM,blackbox,kalman_filter;linear;rc_model;state_space,...,NM,NM,NM,NM,air_temperature (10),concentration (6),humidity (16),occupancy (5),return_air (17);supply_air (12);water_flow (1),pmv (92)
1,10.1016_j.apenergy.2020.115147.json,building (60);temperature (11);room (9);zone (1),acmv (14);accuracy (7);achieve (6);chiller (6)...,NM,occupancy (6);internal (5);heat_loads (1),blackbox,blackbox,blackbox,blackbox,ann;linearized,...,NM,NM,performance_analysis (1),kw (12),air_temperature (35),concentration (5),humidity (13),occupancy (7),supply_air (21);damper (3);return_air (3),pmv (150)
2,10.1016_j.apenergy.2020.115426.json,room (21);temperature (9),ac (11),NM,NM,blackbox,blackbox,NM,NM,bcnn;convolutional_neural_networks,...,demand_response (2),NM,NM,NM,NM,NM,NM,NM,NM,NM
3,10.1016_j.apenergy.2021.117276.json,building (9);temperature (2),achieved (26);actual (22);accurate (11);accura...,NM,NM,blackbox,blackbox,blackbox,blackbox,data_driven_approach,...,NM,NM,performance_evaluation (2),kwh (62),NM,NM,humidity (3);wind_speed (3);weather_station (2),presence (1),NM,NM
4,10.1016_j.apenergy.2021.117987.json,room (46);temperature (7);air_quality (3);buil...,accuracy (36);coil (11);hvac (8);according (2)...,outdoor_temperature (8),occupancy (46);internal (7),NM,NM,NM,NM,NM,...,NM,commissioning (4),performance_evaluation (2),kw (2),air_temperature (19),NM,outdoor_temperature (13);humidity (6),occupancy (107),return_air (12);damper (1),NM
5,10.1016_j.apenergy.2022.119580.json,zone (61);temperature (40);room (31);building ...,achieved (8);according (7);actual (7);accuracy...,outdoor_temperature (2),internal (20);occupant_number (4);infiltration...,greybox,greybox,NM,greybox,rc_model,...,NM,NM,NM,NM,NM,co2 (32),outdoor_temperature (4);humidity (2),occupancy (1),supply_air (2),pmv (32)
6,10.1016_j.buildenv.2020.107089.json,room (17);temperature (8);zone (4);building (2),actuation (4);vav (3);according (2);accuracy (...,NM,occupancy_patterns (2);height (1),NM,NM,NM,NM,NM,...,NM,NM,NM,kwh (2),NM,NM,solar_irradiance (2);humidity (1),occupancy (6),air_flow (14);supply_air (1),NM
7,10.1016_j.enbuild.2020.109792.json,temperature (4);building (3);room (3),accuracy (1);accurate (1);hvac (1);valve (1),NM,NM,NM,NM,NM,NM,NM,...,NM,NM,NM,kwh (2),NM,NM,NM,NM,NM,NM
8,10.1016_j.enbuild.2020.110271.json,building (49);ieq (12);temperature (12);zone (...,actual (28);across (6);account (5);accuracy (2...,NM,occupancy (2),blackbox,blackbox,NM,NM,statistical,...,NM,commissioning (5),NM,NM,air_temperature (2),concentration (4),NM,occupancy (14),NM,NM
9,10.1016_j.enbuild.2020.110276.json,building (71),actual (51);pump (26);chiller (24);ahu (16);hv...,NM,occupancy (1),blackbox,NM,NM,NM,statistical_analysis,...,NM,commissioning (6),NM,kw (14),air_temperature (4),NM,outdoor_temperature (6),occupancy (11),supply_air (10);return_air (6);air_flow (2);da...,NM
